In [ ]:
using AutoGrad
using Deeplearning
using Statistics
using StatsBase
using Plots
import CSV

In [ ]:
batch = 100
learning_rate = 10e-2

In [ ]:
# test = CSV.read("datasets/digit-recognizer/test.csv")

In [ ]:
train = CSV.read("datasets/digit-recognizer/train.csv")
xtrain = []
ytrain = []
for i in 1:size(train)[1]
    max = maximum(train[i,2:end])+0.1
    normalized = reshape(train[i, 2:end]./(max), (28,28,1,1))
    push!(xtrain, normalized)
    push!(ytrain, train[i,1]+1)
end
println("Ready...")

In [ ]:
input = @cudaarray rand(28, 28, 1, 10)
w1 = ((rand(3,3,1,6).-0.5)./100)
w2 = ((rand(3,3,6,16).-0.5)./100)
println("Ready...")

In [ ]:
conv1 = convolve(input, w1, (1,1), (1,1))
x = conv1(input)
pool1 = avgpooling(x, (2,2), (2,2), (1,1))
x = pool1(x)
conv2 = convolve(x, w2, (1,1), (1,1))
x = conv2(x)
pool2 = avgpooling(x, (2,2), (2,2), (1,1))
x = pool2(x)
dense1 = dense(x, 120)
x = dense1(x)
dense2 = dense(x, 84)
x = dense2(x)
dense3 = dense(x, 10)
layers=[dense3,dense2,dense1,conv2,conv1]
println("Loaded")

In [ ]:
function f(x)
    x = conv1(x)
    x = relu(x)
    x = pool1(x)
    
    x = conv2(x)
    x = relu(x)
    x = pool2(x)
    
    x = dense1(x)
    x = relu(x)
    x = dense2(x)
    x = relu(x)
    x = dense3(x)
    x = softmax(x)
end

In [ ]:
acc_train_plot = []
acc_test_plot = []
loss_train_plot = []
loss_test_plot = []

In [ ]:
dff(x,y) = @diff nll(f(x), y)

for e in 1:1
    println("Epoch ", ei)
    indices = sample(1:size(train)[1], size(train)[1], replace = false)
    for (ei,si) in enumerate(1:batch:size(indices)[1])
        sample_batch_x, sample_batch_y = cat(xtrain[indices[(ei):(ei-1)+batch]]..., dims=4), ytrain[indices[(ei):(ei-1)+batch]]

        dv = dff(sample_batch_x, sample_batch_y)

        dparams = @parameters dv

        for pidx in 1:length(dparams[1:end-1])
            dw = grad(dv, dparams[pidx])
            layers[pidx].matrix .-=0.1*dw
        end
    end
    otrain = f(xtrain)
    otest = f(xtest)
    push!(acc_train_plot, acc(o, ytrain))
    push!(loss_train_plot, nll(o, ytrain))
    push!(acc_test_plot, acc(o, ytest))
    push!(loss_test_plot, nll(o, ytest))
end

In [ ]:
plot([acc_plot], linewidth=2, title="Accuracy")

In [ ]:
plot([loss_plot], linewidth=2, title="Loss")